### Step 1: Loading and Preprocessing Data

**Loading Libraries:**
We import necessary libraries such as Pandas for data manipulation, OpenAI for utilizing the GPT-3 model, JSON for handling JSON data, and dotenv for loading environment variables.

In [2]:
import pandas as pd
import openai
import json
from dotenv import dotenv_values

**Loading API Key:** We load the OpenAI API key from the environment variables stored in a `.env` file.

In [3]:
config = dotenv_values(".env")

In [30]:
openai.api_key = config["OPEN_API_KEY"]

**Defining Function for Description Extraction:** We define a function `get_description_details` to extract specific details from product titles and descriptions using the OpenAI GPT-3 model.

In [31]:
def get_description_details(title, description):
    respon_gpt = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user",
        "content": f"""
        {title}
        {description}

        Please provide me with these points based on the title and description above, and fill them according to their format. Make all the languages English, including the keys, and format them as a dictionary with keys and values. If the data is not available, just use a '-' dash

        Brand (Category)
        Material casing (Category)
        Underglow RGB (Yes/No)
        South-facing RGB switch (Yes/No)
        Hot-swappable (5 pin) (Yes/No)
        Compatibility with mechanical switch (Category)
        Magic FN key (Yes/No)
        Software customization (Yes/No)
        Keycap profile (Category)
        Color options (Category)
        Switch options (Category)
        Keyboard layout (Category)
        Number of keys (Number)
        Dimensions (Length) (millimeters)
        Dimensions (Width) (millimeters)
        Dimensions (Height) (millimeters)
        Weight (grams)
        Pre-lubed stabilizer (Yes/No)
        Include damper foam (Yes/No)
        Detachable Type-C (Yes/No)
        Onboard memory (Yes/No)
        Anti-ghosting (Yes/No)
        RGB Per-Key (Yes/No)
        Battery capacity (mAh)
        """}
        ],
        # max_tokens=350, 
        max_tokens=700 # for title_error
    )

    response = respon_gpt.choices[0].message.content

    json_start = response.find('{')
    json_end = response.rfind('}') + 1

    # Periksa apakah respons berupa JSON yang valid
    if json_start != -1 and json_end != -1:
        response_str = response[json_start:json_end]
        
        # Coba parse JSON
        try:
            response_dict = json.loads(response_str)
            return response_dict
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}. Skipping to next row.")
            return None
    else:
        print("No valid JSON found in response. Skipping to next row.")
        return None

**Loading Dataset:** We read the dataset containing mechanical keyboard information from a CSV file scraped data into a Pandas DataFrame. After that, we iterate through each row in the DataFrame to process the descriptions and extract relevant details using the defined function.

In [33]:
# Baca file CSV
df = pd.read_csv('D:\Career\QWork\scrapy\\bukalapak\mechanical-keboard-1.csv')

# Hapus baris yang tidak memiliki judul
df.dropna(subset=['Title'], inplace=True)

# Iterasi setiap baris
for index, row in df.iterrows():
    title = row['Title']
    description = row['Description']
    
    # Proses deskripsi untuk mendapatkan poin-poin
    # max_tokens = 350
    details = get_description_details(title, description)

    # Jika details adalah None, cetak nomor baris
    if details is None:
        print(f"Skipping row {index + 1} - Title: {title}")
        continue
    
    # Update dataframe dengan poin-poin yang dihasilkan
    for key, value in details.items():
        df.at[index, key] = value

    # Print pesan untuk setiap item yang merupakan kelipatan 50
    if (index + 1) % 50 == 0:
        print(f"Item {index+1} processed")

print(f"Total {index + 1} rows processed")

Error decoding JSON: Expecting ',' delimiter: line 17 column 45 (char 634). Skipping to next row.
Skipping row 30 - Title: Keychron K3 v2 Gateron Low Profile WHITE Wireless Mechanical Keyboard 
Error decoding JSON: Expecting ',' delimiter: line 17 column 45 (char 565). Skipping to next row.
Skipping row 33 - Title: Keychron K6 Non Backlight Aluminum Frame Wireless Mechanical Keyboard 
Item 150 processed
Error decoding JSON: Expecting ',' delimiter: line 15 column 31 (char 535). Skipping to next row.
Skipping row 164 - Title: Rexus Keyboard Numeric Mechanical Daxa M22 
Error decoding JSON: Expecting property name enclosed in double quotes: line 26 column 1 (char 772). Skipping to next row.
Skipping row 175 - Title: Keyboard Gaming Logitech G413 SE TKL Mechanical Tactile Backlit 
Item 200 processed
Item 250 processed
Item 300 processed
Item 350 processed
Item 400 processed
Error decoding JSON: Expecting ',' delimiter: line 15 column 49 (char 639). Skipping to next row.
Skipping row 410 -

In [34]:
# Simpan dataframe ke file CSV baru
df.to_csv('mechanical-keyboard-processed.csv', index=False)

**Merging Duplicate Columns:** We identify and merge columns with duplicate information to streamline the dataset. This includes merging columns related to dimensions and weight.

In [36]:
# Kolom-kolom yang akan digabungkan
columns_to_merge = [
    ("Brand (Category)", "Brand"),
    ("Material casing (Category)", "Material casing"),
    ("Underglow RGB (Yes/No)", "Underglow RGB"),
    ("South-facing RGB switch (Yes/No)", "South-facing RGB switch"),
    ("Hot-swappable (5 pin) (Yes/No)", "Hot-swappable (5 pin)"),
    ("Compatibility with mechanical switch (Category)", "Compatibility with mechanical switch"),
    ("Magic FN key (Yes/No)", "Magic FN key"),
    ("Software customization (Yes/No)", "Software customization"),
    ("Keycap profile (Category)", "Keycap profile"),
    ("Color options (Category)", "Color options"),
    ("Switch options (Category)", "Switch options"),
    ("Keyboard layout (Category)", "Keyboard layout"),
    ("Number of keys (Number)", "Number of keys"),
    ("Dimensions (Length) (millimeters)", "Dimensions (Length)"),
    ("Dimensions (Width) (millimeters)", "Dimensions (Width)"),
    ("Dimensions (Height) (millimeters)", "Dimensions (Height)"),
    ("Weight (grams)", "Weight"),
    ("Pre-lubed stabilizer (Yes/No)", "Pre-lubed stabilizer"),
    ("Include damper foam (Yes/No)", "Include damper foam"),
    ("Detachable Type-C (Yes/No)", "Detachable Type-C"),
    ("Onboard memory (Yes/No)", "Onboard memory"),
    ("Anti-ghosting (Yes/No)", "Anti-ghosting"),
    ("RGB Per-Key (Yes/No)", "RGB Per-Key"),
    ("Battery capacity (mAh)", "Battery capacity")
]

# Melakukan merge kolom-kolom yang duplikat
for col1, col2 in columns_to_merge:
    df[col1].fillna(df[col2], inplace=True)
    df.drop(columns=[col2], inplace=True)

In [37]:
df

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh),Dimensions (Length) (mm),Dimensions (Width) (mm),Dimensions (Height) (mm),Weight (g),Supported Systems,Include dampener foam,Dimensions (Height)(millimeters)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,NaN,NaN,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6.516 Feedback Positif,dari 6.541 feedback,...,Yes,No,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,NaN,NaN,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1.216 Feedback Positif,dari 1.275 feedback,...,Yes,Yes,3750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,NaN,NaN,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3.132 Feedback Positif,dari 3.136 feedback,...,Yes,No,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,NaN,NaN,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4.253 Feedback Positif,dari 4.479 feedback,...,Yes,Yes,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,NaN,NaN,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5.693 Feedback Positif,dari 5.940 feedback,...,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,BEST SELLER 1STPLAYER Mechanical Gaming Keyboa...,NaN,NaN,Rp1.425.000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3 Feedback Positif,dari 6 feedback,...,Yes,No,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2124,PROMO Powerlogic Armageddon MKA 9C RGB Mechani...,NaN,NaN,Rp1.185.000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3 Feedback Positif,dari 6 feedback,...,Yes,No,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2125,Terunik Powerlogic Armageddon MKA 5R RGB Mecha...,NaN,NaN,Rp1.440.000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3 Feedback Positif,dari 6 feedback,...,Yes,No,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2126,Terunik Powerlogic Armageddon MKA 11R RGB Mech...,NaN,NaN,Rp1.612.500,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3 Feedback Positif,dari 6 feedback,...,Yes,No,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Kolom-kolom yang akan digabungkan
columns_to_merge = [
    ("Dimensions (Length) (millimeters)", "Dimensions (Length) (mm)"),
    ("Dimensions (Width) (millimeters)", "Dimensions (Width) (mm)"),
    ("Dimensions (Height) (millimeters)", "Dimensions (Height) (mm)"),
    ("Weight (grams)", "Weight (g)"),
    ("Include damper foam (Yes/No)", "Include dampener foam")
]

# Melakukan merge kolom-kolom yang duplikat
for col1, col2 in columns_to_merge:
    df[col1].fillna(df[col2], inplace=True)
    df.drop(columns=[col2], inplace=True)

In [40]:
df.head()

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh),Supported Systems,Dimensions (Height)(millimeters)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,NaN,NaN,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6.516 Feedback Positif,dari 6.541 feedback,...,950.0,No,No,No,Yes,Yes,No,-,NaN,NaN
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,NaN,NaN,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1.216 Feedback Positif,dari 1.275 feedback,...,800.0,No,No,Yes,No,Yes,Yes,3750,NaN,NaN
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,NaN,NaN,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3.132 Feedback Positif,dari 3.136 feedback,...,-,No,No,Yes,No,Yes,No,1850,NaN,NaN
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,NaN,NaN,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4.253 Feedback Positif,dari 4.479 feedback,...,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-,NaN,NaN
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,NaN,NaN,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5.693 Feedback Positif,dari 5.940 feedback,...,-,-,-,-,-,-,-,-,NaN,NaN


In [42]:
# Kolom-kolom yang akan digabungkan
columns_to_merge = [
    ("Dimensions (Height) (millimeters)", "Dimensions (Height)(millimeters)")
]

# Melakukan merge kolom-kolom yang duplikat
for col1, col2 in columns_to_merge:
    df[col1].fillna(df[col2], inplace=True)
    df.drop(columns=[col2], inplace=True)

In [43]:
# Menghapus kolom "Supported Systems"
df.drop(columns=["Supported Systems"], inplace=True)

In [44]:
# Simpan dataframe ke file CSV baru
df.to_csv('mechanical-keyboard-processed-merged.csv', index=False)

In [179]:
df = pd.read_csv("D:\Career\QWork\chatgpt\mechanical-keyboard-processed-merged.csv")

In [180]:
df.head()

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,NaN,NaN,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6.516 Feedback Positif,dari 6.541 feedback,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,-
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,NaN,NaN,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1.216 Feedback Positif,dari 1.275 feedback,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,NaN,NaN,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3.132 Feedback Positif,dari 3.136 feedback,...,102.0,40.0,-,No,No,Yes,No,Yes,No,1850
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,NaN,NaN,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4.253 Feedback Positif,dari 4.479 feedback,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,NaN,NaN,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5.693 Feedback Positif,dari 5.940 feedback,...,-,-,-,-,-,-,-,-,-,-


### Step 2: Data Cleaning and Preparation

**Removing Rows with Missing Titles:** We remove rows with missing attributes as they are crucial for the description extraction process.

In [181]:
# Count the number of rows before removal
total_rows_before = df.shape[0]

# Remove rows with empty values in the 'Brand (Category)' column
df.dropna(subset=['Brand (Category)'], inplace=True)

# Count the number of rows after removal
total_rows_after = df.shape[0]

# Calculate the number of rows removed
rows_removed = total_rows_before - total_rows_after

print(f"Number of rows removed: {rows_removed}")

Number of rows removed: 43


**Cleaning Feedback Columns:** We clean the 'Positive Feedback' and 'Total Feedback' columns, removing unnecessary text and converting them to integer values.

In [182]:
import re

# Function to clean the feedback string
def clean_feedback(feedback):
    if isinstance(feedback, str):
        # Remove dots
        feedback = feedback.replace('.', '')
        # Remove " Feedback Positif"
        feedback = re.sub(r'\sFeedback\sPositif', '', feedback)
    return feedback

In [183]:
# Apply the function to the entire 'Positive Feedback' column
df['Positive Feedback'] = df['Positive Feedback'].apply(lambda x: clean_feedback(x))

In [184]:
df.head()

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,NaN,NaN,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,dari 6.541 feedback,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,-
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,NaN,NaN,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,dari 1.275 feedback,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,NaN,NaN,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,dari 3.136 feedback,...,102.0,40.0,-,No,No,Yes,No,Yes,No,1850
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,NaN,NaN,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,dari 4.479 feedback,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,NaN,NaN,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,dari 5.940 feedback,...,-,-,-,-,-,-,-,-,-,-


In [185]:
# Function to clean the total feedback string
def clean_total_feedback(feedback):
    if isinstance(feedback, str):
        # Mencocokkan angka dalam string menggunakan ekspresi reguler
        numeric_part = re.search(r'[\d,\.]+', feedback)
        if numeric_part:
            numeric_value = int(numeric_part.group().replace(',', '').replace('.', ''))
            return numeric_value
    return None  # Mengembalikan None jika tidak ada angka yang ditemukan

In [186]:
# Apply the function to the entire 'Positive Feedback' column
df['Total Feedback'] = df['Total Feedback'].apply(lambda x: clean_total_feedback(x))

In [187]:
df.head()

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,NaN,NaN,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,6541.0,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,-
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,NaN,NaN,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,1275.0,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,NaN,NaN,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,3136.0,...,102.0,40.0,-,No,No,Yes,No,Yes,No,1850
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,NaN,NaN,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,4479.0,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,NaN,NaN,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,5940.0,...,-,-,-,-,-,-,-,-,-,-


In [188]:
# Fill NaN values with 0 in 'Positive Feedback' and 'Total Feedback' columns
df['Positive Feedback'] = df['Positive Feedback'].fillna(0)
df['Total Feedback'] = df['Total Feedback'].fillna(0)

# Convert 'Positive Feedback' and 'Total Feedback' columns to integers
df['Positive Feedback'] = df['Positive Feedback'].astype(int)
df['Total Feedback'] = df['Total Feedback'].astype(int)

In [189]:
df.head()

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,NaN,NaN,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,6541,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,-
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,NaN,NaN,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,1275,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,NaN,NaN,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,3136,...,102.0,40.0,-,No,No,Yes,No,Yes,No,1850
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,NaN,NaN,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,4479,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,NaN,NaN,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,5940,...,-,-,-,-,-,-,-,-,-,-


**Cleaning Numeric Columns:** We clean and convert various numeric columns like 'Original Price', 'Reviews', and 'Sold' to appropriate data types.

In [190]:
# Remove the substring " Ulasan" from non-empty values in 'Reviews' column
df['Reviews'] = df['Reviews'].str.replace(' Ulasan', '')

In [191]:
# Fill NaN values in 'Reviews' column with 0
df['Reviews'] = df['Reviews'].fillna(0)

In [192]:
# Convert 'Reviews' column to integer data type
df['Reviews'] = df['Reviews'].astype(int)

In [193]:
# Remove the substring " Terjual" from non-empty values in 'Sold' column
df['Sold'] = df['Sold'].str.replace(' Terjual', '')

In [194]:
# Fill NaN values in 'Sold' column with 0
df['Sold'] = df['Sold'].fillna(0)

In [196]:
df

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,0,0,Rp549.000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,6541,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,-
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,0,0,Rp1.049.000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,1275,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,0,0,Rp538.700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,3136,...,102.0,40.0,-,No,No,Yes,No,Yes,No,1850
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,0,0,Rp518.400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,4479,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,0,0,Rp79.000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,5940,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,BEST SELLER 1STPLAYER Mechanical Gaming Keyboa...,0,0,Rp1.425.000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,230,45,1400,No,No,No,No,Yes,No,-
1860,PROMO Powerlogic Armageddon MKA 9C RGB Mechani...,0,0,Rp1.185.000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,-,-,-,No,No,No,No,Yes,No,-
1861,Terunik Powerlogic Armageddon MKA 5R RGB Mecha...,0,0,Rp1.440.000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,146,38,-,No,No,No,No,Yes,No,-
1862,Terunik Powerlogic Armageddon MKA 11R RGB Mech...,0,0,Rp1.612.500,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,-,-,-,No,No,No,Yes,Yes,No,-


In [198]:
df.to_csv("test-3.csv", index=False)

In [306]:
df = pd.read_csv('test-3.csv')

In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 35 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Title                                            1821 non-null   object 
 1   Reviews                                          1821 non-null   int64  
 2   Sold                                             1821 non-null   float64
 3   Original Price                                   1821 non-null   object 
 4   Store Name                                       1821 non-null   object 
 5   Store City                                       1821 non-null   object 
 6   Description                                      1821 non-null   object 
 7   Order Processing                                 1802 non-null   object 
 8   Positive Feedback                                1821 non-null   int64  
 9   Total Feedback                

In [308]:
# Remove non-numeric characters from 'Original Price' column and convert to integer
df['Original Price'] = df['Original Price'].str.replace('Rp', '').str.replace('.', '').astype(int)

**Standardizing Brand Names:** We standardize brand names using a predefined dictionary to ensure consistency.

In [309]:
unique_brands = df['Brand (Category)'].unique()
print(unique_brands)


['Rexus (Gaming Mechanical Keyboard)' 'Rexus' 'PXN' 'Fantech' 'Gamen'
 'Redragon' 'Leopold' 'Ducky' 'Tecware' 'Cooler Master (Gaming Keyboard)'
 'AULA' 'POWERUP RAPTOR' 'Thermaltake' 'Keycaps Gundam Theme'
 'Rexus (Gaming Keyboard)' 'Clover Garment Indonesia' 'Rexus Daxa'
 'Logitech' 'Digital Alliance' 'Cooler Master' 'Keychron' 'Aukey' 'Daxa'
 'Gateron' 'NYK' 'Gamen Titan IV' 'ZHUIGUANGBAO' 'Altec Lansing' 'Aula'
 'Logitech G413' 'Rapoo' 'VODOOL' 'GAMEN' 'Armaggeddon' 'Sades' 'Cherry'
 'Royal Kludge' 'ZIYOULANG' 'NB' 'Keycaps mechanical' 'HyperX' 'Imperion'
 'WORKMATE II' 'Paradox Gaming' 'Noxelic' 'Leopard' 'MSI' 'Rexus MX5.2'
 'Keycaps Gundam Theme Set' 'NOIR' 'SADES' 'Gamdias'
 'Razer (Gaming Keyboard)' 'Rexus Daiva' 'Tombol' 'FOOMEE' 'Leopard G20'
 'CYBORG' 'Logitech G413SE' 'Keycaps CLV' 'HAVIT' 'Kingston HyperX'
 'TOMBOL' 'K68' 'COOLER MASTER' 'Raptor' 'COUGAR' 'LOGITECH'
 'Redragon (Gaming Keyboard)' 'VD12' 'Paradox Gaming Ghost' 'DAREU' '-'
 'Shortcut' 'Rexus MX9' 'Skyloong' '

In [310]:
len(unique_brands)

356

In [311]:
import re

# Dictionary untuk mapping merek
brand_replacements = {
    'Rexus': 'Rexus',
    'Logitech': 'Logitech',
    'Tecware': 'Tecware',
    'NYK': 'NYK',
    'SteelSeries': 'SteelSeries',
    'MSI': 'MSI',
    'AULA': 'AULA',
    'Lenovo': 'Lenovo',
    'HyperX': 'HyperX',
    'E-YOOSO': 'E-YOOSO',
    'AJAZZ': 'AJAZZ',
    'Ducky': 'Ducky',
    'Cooler Master': 'Cooler Master',
    'Fantech': 'Fantech',
    'ASUS': 'ASUS',
    'Azio': 'Azio',
    'REDRAGON': 'REDRAGON',
    'Razer': 'Razer',
    'Rapoo': 'Rapoo',
    'Vintage': 'Vintage',
    'Royal Kludge': 'Royal Kludge',
    'Vortex': 'Vortex',
    'Keycap Puller': 'Keycap Puller',
    'COUGAR': 'COUGAR',
    'Powerlogic': 'Powerlogic',
    'AXE': 'AXE',
    'FiiO': 'FiiO',
    'Philips': 'Philips',
    'Noir': 'Noir'
}

# Fungsi untuk mengganti merek
def replace_brand(brand):
    for key, value in brand_replacements.items():
        if re.search(rf'\b{re.escape(key)}\b', brand, re.IGNORECASE):
            return value
    return brand

# Mengganti merek menggunakan fungsi replace_brand
df['Brand (Category)'] = df['Brand (Category)'].apply(replace_brand)


In [312]:
unique_brands = df['Brand (Category)'].unique()
print(unique_brands)

['Rexus' 'PXN' 'Fantech' 'Gamen' 'REDRAGON' 'Leopold' 'Ducky' 'Tecware'
 'Cooler Master' 'AULA' 'POWERUP RAPTOR' 'Thermaltake'
 'Keycaps Gundam Theme' 'Clover Garment Indonesia' 'Logitech'
 'Digital Alliance' 'Keychron' 'Aukey' 'Daxa' 'Gateron' 'NYK'
 'Gamen Titan IV' 'ZHUIGUANGBAO' 'Altec Lansing' 'Rapoo' 'VODOOL' 'GAMEN'
 'Armaggeddon' 'Sades' 'Cherry' 'Royal Kludge' 'ZIYOULANG' 'NB'
 'Keycaps mechanical' 'HyperX' 'Imperion' 'WORKMATE II' 'Paradox Gaming'
 'Noxelic' 'Leopard' 'MSI' 'Keycaps Gundam Theme Set' 'Noir' 'SADES'
 'Gamdias' 'Razer' 'Tombol' 'FOOMEE' 'Leopard G20' 'CYBORG' 'Keycaps CLV'
 'HAVIT' 'TOMBOL' 'K68' 'Raptor' 'COUGAR' 'VD12' 'Paradox Gaming Ghost'
 'DAREU' '-' 'Shortcut' 'Skyloong' 'Remax' 'Dareu'
 'Originale Leopard G700' 'ASUS' 'Clover Gaming Indonesia' 'CHERRY'
 'Predator' 'DELUXE' 'Y-FRUITFUL' 'DragonWar' 'Komic' 'MARVO' 'HP' 'ALTEC'
 'Leaven' 'MAXFIT67' 'FIREROSE Gaming' 'VD6R' 'HELENS MKN-06' 'VD6B' 'PBT'
 'Combo' 'Robot Workmate II' 'GAMING DA' '5STAR' 'Viva

In [313]:
len(unique_brands)

259

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 35 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Title                                            1821 non-null   object 
 1   Reviews                                          1821 non-null   int64  
 2   Sold                                             1821 non-null   float64
 3   Original Price                                   1821 non-null   int32  
 4   Store Name                                       1821 non-null   object 
 5   Store City                                       1821 non-null   object 
 6   Description                                      1821 non-null   object 
 7   Order Processing                                 1802 non-null   object 
 8   Positive Feedback                                1821 non-null   int64  
 9   Total Feedback                

In [315]:
unique_materials = df['Material casing (Category)'].unique()
print(unique_materials)

['Plastik ABS' 'ABS' 'Plastic' 'Polycarbonate' '-'
 'Plastic body / Gunmetal Alloy Backplate' 'Plastic ABS'
 'Plastic, Aluminum' 'Metal' 'PBT' 'High Quality ABS'
 'Strong, corrosion-resistant material with smooth texture'
 'Aluminum, Plastic' 'Metal Cover ABS' 'Aluminum, ABS' 'Mechanical'
 'ABS Plastic' 'Sandblasted Aluminum, Plastic' 'Aluminum' 'Aluminum alloy'
 'Acrylic and Plastic' 'FR4 Plate' 'PET mesh and braided cable'
 'ABS Double Injection' 'Kevlar'
 'Black Plastic Body / Gunmetal Alloy Backplate' 'ABS PA-758'
 'PBT double injection' 'Aluminium plate'
 'Strong, corrosion-resistant material'
 'Resin, CNC Aluminium, CNC Acrylic' 'Top Cover Metal, Bottom Cover ABS'
 'Silicon and Polycarbonate' 'PBT Dye Sub' 'Brushed Aluminum'
 'Metal (Top Cover), ABS (Bottom Cover)'
 'Plastic ABS with aluminum alloy cover' 'Metal Plate' 'ABS + Metal Cover'
 'Transparent' 'Aluminum Alloy' 'Full aircraft-grade aluminum body'
 'Carbon' 'Plastic, Aluminum, PU Leather' 'Metal and ABS' 'Acrylic'
 'Gunme

In [316]:
len(unique_materials)

269

In [317]:
unique_underglow = df['Underglow RGB (Yes/No)'].unique()
print(unique_underglow)

['No' 'Yes' '-']


In [318]:
unique_south_facing = df['South-facing RGB switch (Yes/No)'].unique()
print(unique_south_facing)

['No' 'Yes' '-']


In [319]:
unique_hot_swap = df['Hot-swappable (5 pin) (Yes/No)'].unique()
print(unique_hot_swap)

['No' 'Yes' '-']


In [320]:
unique_compatibility_mech_swith = df['Compatibility with mechanical switch (Category)'].unique()
print(unique_compatibility_mech_swith)

['-' 'Gateron' 'Cherry MX'
 'Gateron, Cherry, Kailh, and other 5-pin mechanical switch types' 'MX'
 'Blue switch' 'Cherry MX Brown' 'Wraith Switches' 'Yes'
 'Outemu Brown Switch' 'Cherry MX RGB Blue'
 'Cherry MX, Outemu, Gateron, etc' 'Red Switch' 'SNK Switch' 'Fantech'
 'Gateron KS-9' 'Romer-G Mechanical Switches' 'Outemu Removable Switch'
 'Outemu Blue Switch' 'Low Profile Mechanical Switches'
 'Gateron, Cherry, Kailh etc.' 'Outemu Switch' 'Outemu Red'
 'TTC Low Profile Mechanical Switch (Red/Blue/Brown)'
 'Cherry MX Blue and Cherry MX Red' 'Swappable switch' 'Blue Switch'
 'Gateron Pro Milky Yellow / Red, JWK T1 (Blue)' 'Cherry MX Silent Red'
 'Outemu' 'SNK Switch (Blue, Red, Brown)' 'Red/Blue/Brown'
 'Tactile Mechanical Switch' 'Swappable 3 pin switch / 5 Pin Switch'
 'Gateron, Cherry, Kailh' 'Outemu Removable Blue Switch' 'Universal'
 'Romer-G' 'Romer-G Tactile/Linear, GX Blue' 'Red switch'
 'Outemu Box Switch' 'Outemu Switches' 'Kailh' 'KTT White, KTT Ash Grey'
 'Fantech Switch' 

In [321]:
len(unique_compatibility_mech_swith)

474

In [322]:
df['Compatibility with mechanical switch (Category)'] = df['Compatibility with mechanical switch (Category)'].replace({'Yes': '-', 'No': '-'})

In [323]:
unique_magic_fn = df['Magic FN key (Yes/No)'].unique()
print(unique_magic_fn)

['No' 'Yes' '-' 'Yes (Hold FN+L_alt+L)']


In [325]:
df['Magic FN key (Yes/No)'] = df['Magic FN key (Yes/No)'].replace('Yes (Hold FN+L_alt+L)','Yes')

In [326]:
unique_magic_fn = df['Magic FN key (Yes/No)'].unique()
print(unique_magic_fn)

['No' 'Yes' '-']


In [327]:
unique_soft_custome = df['Software customization (Yes/No)'].unique()
print(unique_soft_custome)

['Yes' 'No' '-' 'Cooler Master MasterPlus+' 'Razer Synapse 3 enabled'
 'QMK & VIA' 'Yes (VIA)' 'Cooler Master MasterPlus+ Software'
 'Razer Synapse enabled' 'HERA Software' 'Yes (Cooler Master MasterPlus+)'
 'Armoury Crate' 'Yes (Armoury Crate)' 'Yes (Razer Synapse)'
 'SteelSeries Engine 3.15+ (coming soon)' 'Support VIA'
 'Yes - 8BitDo Ultimate Software V2'
 'Logitech Options for Windows 10 or later, Logitech Options for macOS 10.8 or later, Logitech Flow'
 'Yes (QMK/VIA)' 'Logi Options+' 'Pro Driver Support' 'QMK/VIA support']


In [328]:
df['Software customization (Yes/No)'] = df['Software customization (Yes/No)'].apply(lambda x: 'Yes' if x not in ['No', '-'] else x)

In [329]:
unique_soft_custome = df['Software customization (Yes/No)'].unique()
print(unique_soft_custome)

['Yes' 'No' '-']


In [330]:
df.to_csv('test-4.csv', index=False)

In [16]:
df = pd.read_csv('test-4.csv')
df

,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,Total Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,0,0.0,549000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,6541,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,-
1,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,0,0.0,1049000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,1275,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,PXN K30 Gaming TKL Battledock Mechanical Keybo...,0,0.0,538700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,3136,...,102.0,40.0,-,No,No,Yes,No,Yes,No,1850
3,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,0,0.0,518400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,4479,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,-
4,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,0,0.0,79000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,5940,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,BEST SELLER 1STPLAYER Mechanical Gaming Keyboa...,0,0.0,1425000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,230,45,1400,No,No,No,No,Yes,No,-
1817,PROMO Powerlogic Armageddon MKA 9C RGB Mechani...,0,0.0,1185000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,-,-,-,No,No,No,No,Yes,No,-
1818,Terunik Powerlogic Armageddon MKA 5R RGB Mecha...,0,0.0,1440000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,146,38,-,No,No,No,No,Yes,No,-
1819,Terunik Powerlogic Armageddon MKA 11R RGB Mech...,0,0.0,1612500,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,6,...,-,-,-,No,No,No,Yes,Yes,No,-


In [17]:
df.columns

Index(['Title', 'Reviews', 'Sold', 'Original Price', 'Store Name',
       'Store City', 'Description', 'Order Processing', 'Positive Feedback',
       'Total Feedback', 'Product URL', 'Brand (Category)',
       'Material casing (Category)', 'Underglow RGB (Yes/No)',
       'South-facing RGB switch (Yes/No)', 'Hot-swappable (5 pin) (Yes/No)',
       'Compatibility with mechanical switch (Category)',
       'Magic FN key (Yes/No)', 'Software customization (Yes/No)',
       'Keycap profile (Category)', 'Color options (Category)',
       'Switch options (Category)', 'Keyboard layout (Category)',
       'Number of keys (Number)', 'Dimensions (Length) (millimeters)',
       'Dimensions (Width) (millimeters)', 'Dimensions (Height) (millimeters)',
       'Weight (grams)', 'Pre-lubed stabilizer (Yes/No)',
       'Include damper foam (Yes/No)', 'Detachable Type-C (Yes/No)',
       'Onboard memory (Yes/No)', 'Anti-ghosting (Yes/No)',
       'RGB Per-Key (Yes/No)', 'Battery capacity (mAh)'],
     

In [18]:
last_columns = ['Pre-lubed stabilizer (Yes/No)',
       'Include damper foam (Yes/No)', 'Detachable Type-C (Yes/No)',
       'Onboard memory (Yes/No)', 'Anti-ghosting (Yes/No)',
       'RGB Per-Key (Yes/No)']

In [19]:
for columns in last_columns:
    unique_data = df[columns].unique()
    print(columns)
    print(unique_data)
    print('----------------------')

Pre-lubed stabilizer (Yes/No)
['No' 'Yes' '-' 'Cherry screw-in stabilizers']
----------------------
Include damper foam (Yes/No)
['No' 'Yes' '-'
 'Poron Plate Foam, IXPE Switch Pad, PCBA, Poron Case Foam, VHB Insulation Layer, Extra Tape Mod, Extra Teflon Pads']
----------------------
Detachable Type-C (Yes/No)
['No' 'Yes' '-']
----------------------
Onboard memory (Yes/No)
['Yes' 'No' '-'
 'Hybrid On-Board Memory and Cloud Storage up to 5 profiles'
 'Hybrid onboard storage up to 5 profiles'
 'Yes (Hybrid onboard storage up to 5 profiles)'
 'Hybrid onboard storage up to 5 keybinding profiles']
----------------------
Anti-ghosting (Yes/No)
['Yes' '-' '26 Keys' 'No' 'All Key' 'Full key anti-ghosting'
 'Yes (68 keys)' '6-key rollover anti-ghosting' 'Full' 'N Key Rollover'
 'Not specified' 'Full Keys Anti-Ghosting' 'N-Key Rollover' 'Yes (NKRO)'
 'N-Key rollover']
----------------------
RGB Per-Key (Yes/No)
['No' 'Yes' '-' 'There are 20 different backlight options' 'Unknown'
 'Rainbow backl

In [20]:
for columns in last_columns:
    df[columns] = df[columns].apply(lambda x: 'Yes' if x not in ['No', '-'] else x)

In [21]:
for columns in last_columns:
    unique_data = df[columns].unique()
    print(columns)
    print(unique_data)
    print('----------------------')

Pre-lubed stabilizer (Yes/No)
['No' 'Yes' '-']
----------------------
Include damper foam (Yes/No)
['No' 'Yes' '-']
----------------------
Detachable Type-C (Yes/No)
['No' 'Yes' '-']
----------------------
Onboard memory (Yes/No)
['Yes' 'No' '-']
----------------------
Anti-ghosting (Yes/No)
['Yes' '-' 'No']
----------------------
RGB Per-Key (Yes/No)
['No' 'Yes' '-']
----------------------


**Handling Missing Values:** We handle missing values in different columns, filling them with appropriate values or NaN where necessary.

In [23]:
df["Number of keys (Number)"] = pd.to_numeric(df["Number of keys (Number)"], errors='coerce').fillna(0).astype(int)

df["Battery capacity (mAh)"] = pd.to_numeric(df["Battery capacity (mAh)"], errors='coerce').fillna(0).astype(int)

df["Weight (grams)"] = pd.to_numeric(df["Weight (grams)"], errors='coerce').fillna(0).astype(float)

# Konversi kolom "Dimensions (Length) (millimeters)" menjadi tipe data float
df["Dimensions (Length) (millimeters)"] = pd.to_numeric(df["Dimensions (Length) (millimeters)"], errors='coerce').fillna(0).astype(float)

# Konversi kolom "Dimensions (Width) (millimeters)" menjadi tipe df float
df["Dimensions (Width) (millimeters)"] = pd.to_numeric(df["Dimensions (Width) (millimeters)"], errors='coerce').fillna(0).astype(float)

# Konversi kolom "Dimensions (Height) (millimeters)" menjadi tipe df float
df["Dimensions (Height) (millimeters)"] = pd.to_numeric(df["Dimensions (Height) (millimeters)"], errors='coerce').fillna(0).astype(float)

In [24]:
df.loc[:, df.dtypes == 'object'] = df.loc[:, df.dtypes == 'object'].replace('-', pd.NA)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 35 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Title                                            1821 non-null   object 
 1   Reviews                                          1821 non-null   int64  
 2   Sold                                             1821 non-null   float64
 3   Original Price                                   1821 non-null   int64  
 4   Store Name                                       1821 non-null   object 
 5   Store City                                       1821 non-null   object 
 6   Description                                      1821 non-null   object 
 7   Order Processing                                 1802 non-null   object 
 8   Positive Feedback                                1821 non-null   int64  
 9   Total Feedback                

In [28]:
# Ganti nilai 0 dengan NaN pada kolom yang dimaksud
cols_to_replace = ['Number of keys (Number)', 'Dimensions (Length) (millimeters)', 'Dimensions (Width) (millimeters)', 
                   'Dimensions (Height) (millimeters)', 'Weight (grams)', 'Battery capacity (mAh)']
df[cols_to_replace] = df[cols_to_replace].replace(0, pd.NA)

In [30]:
df.to_csv('mechanical-keyboard-cleaned-ready.csv', index= False)

**Adding IDs:** We add unique product IDs and store IDs to the dataframe to facilitate further analysis and database organization.

In [31]:
# Tambahkan kolom ID dengan nomor urutan di paling kiri DataFrame
df.insert(0, 'product_id', range(1, len(df) + 1))

In [32]:
df

,product_id,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,...,Dimensions (Width) (millimeters),Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh)
0,1,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,0,0.0,549000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,...,133.5,22.5,950.0,No,No,No,Yes,Yes,No,<NA>
1,2,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,0,0.0,1049000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,...,126.0,41.5,800.0,No,No,Yes,No,Yes,Yes,3750
2,3,PXN K30 Gaming TKL Battledock Mechanical Keybo...,0,0.0,538700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,...,102.0,40.0,<NA>,No,No,Yes,No,Yes,No,1850
3,4,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,0,0.0,518400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,...,103.0,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,<NA>
4,5,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,0,0.0,79000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,1817,BEST SELLER 1STPLAYER Mechanical Gaming Keyboa...,0,0.0,1425000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,230.0,45.0,1400.0,No,No,No,No,Yes,No,NaN
1817,1818,PROMO Powerlogic Armageddon MKA 9C RGB Mechani...,0,0.0,1185000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,<NA>,<NA>,<NA>,No,No,No,No,Yes,No,NaN
1818,1819,Terunik Powerlogic Armageddon MKA 5R RGB Mecha...,0,0.0,1440000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,146.0,38.0,<NA>,No,No,No,No,Yes,No,NaN
1819,1820,Terunik Powerlogic Armageddon MKA 11R RGB Mech...,0,0.0,1612500,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,<NA>,<NA>,<NA>,No,No,No,Yes,Yes,No,NaN


In [35]:
store_id_map = {}
unique_id = 1
for store in df['Store Name'].unique():
    store_id_map[store] = unique_id
    unique_id += 1

# Buat kolom store_id berdasarkan kamus yang telah dibuat
df['store_id'] = df['Store Name'].map(store_id_map)

In [36]:
df

,product_id,Title,Reviews,Sold,Original Price,Store Name,Store City,Description,Order Processing,Positive Feedback,...,Dimensions (Height) (millimeters),Weight (grams),Pre-lubed stabilizer (Yes/No),Include damper foam (Yes/No),Detachable Type-C (Yes/No),Onboard memory (Yes/No),Anti-ghosting (Yes/No),RGB Per-Key (Yes/No),Battery capacity (mAh),store_id
0,1,Keyboard Gaming Mechanical Rexus MX9 TKL Legio...,0,0.0,549000,uneed shop,Jakarta Utara,Rexus Keyboard Gaming Mechanical Legionare MX9...,± 21 jam,6516,...,22.5,950.0,No,No,No,Yes,Yes,No,<NA>,1
1,2,Rexus Daxa M84 Ultimate Hotswap Wireless Mecha...,0,0.0,1049000,Gaming Distro,Jakarta Pusat,"Hi, Welcome to GAMING DISTRO - Authorized Deal...",± 1-2 hari,1216,...,41.5,800.0,No,No,Yes,No,Yes,Yes,3750,2
2,3,PXN K30 Gaming TKL Battledock Mechanical Keybo...,0,0.0,538700,Store Pinokio,Kab. Tangerang,Fitur:61 KEYS MINI KEYBOARD: Meskipun memperke...,± 19 jam,3132,...,40.0,<NA>,No,No,Yes,No,Yes,No,1850,3
3,4,Fantech MK857 Maxfit61 Frost Wired RGB 60% Mec...,0,0.0,518400,Distributor_Aksesoris,Jakarta Pusat,POLYCARBONATE COVER & TRANSPARENT CASING + UND...,± 15 jam,4253,...,42.0,566.0,Yes,Yes,Yes,Yes,Yes,Yes,<NA>,4
4,5,Gamen GMK01 PBT Keycaps - keycaps Mechanical k...,0,0.0,79000,Starcomp,Sleman,Keunggulan:Warna lucu desain manis senada deng...,± 15 jam,5693,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,1817,BEST SELLER 1STPLAYER Mechanical Gaming Keyboa...,0,0.0,1425000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,45.0,1400.0,No,No,No,No,Yes,No,NaN,149
1817,1818,PROMO Powerlogic Armageddon MKA 9C RGB Mechani...,0,0.0,1185000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,<NA>,<NA>,No,No,No,No,Yes,No,NaN,149
1818,1819,Terunik Powerlogic Armageddon MKA 5R RGB Mecha...,0,0.0,1440000,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,38.0,<NA>,No,No,No,No,Yes,No,NaN,149
1819,1820,Terunik Powerlogic Armageddon MKA 11R RGB Mech...,0,0.0,1612500,Nelson,Jakarta Barat,MOHON TANYAKAN TERLEBIH DAHULU UNTUK INFO STOK...,± 1-2 hari,3,...,<NA>,<NA>,No,No,No,Yes,Yes,No,NaN,149


In [37]:
df.columns

Index(['product_id', 'Title', 'Reviews', 'Sold', 'Original Price',
       'Store Name', 'Store City', 'Description', 'Order Processing',
       'Positive Feedback', 'Total Feedback', 'Product URL',
       'Brand (Category)', 'Material casing (Category)',
       'Underglow RGB (Yes/No)', 'South-facing RGB switch (Yes/No)',
       'Hot-swappable (5 pin) (Yes/No)',
       'Compatibility with mechanical switch (Category)',
       'Magic FN key (Yes/No)', 'Software customization (Yes/No)',
       'Keycap profile (Category)', 'Color options (Category)',
       'Switch options (Category)', 'Keyboard layout (Category)',
       'Number of keys (Number)', 'Dimensions (Length) (millimeters)',
       'Dimensions (Width) (millimeters)', 'Dimensions (Height) (millimeters)',
       'Weight (grams)', 'Pre-lubed stabilizer (Yes/No)',
       'Include damper foam (Yes/No)', 'Detachable Type-C (Yes/No)',
       'Onboard memory (Yes/No)', 'Anti-ghosting (Yes/No)',
       'RGB Per-Key (Yes/No)', 'Battery ca

**Creating Separate DataFrames:** We create separate dataframes for stores and products, containing relevant information.

In [43]:
# Membuat DataFrame untuk tabel Stores
stores_df = df[['store_id', 'Store Name', 'Store City']].drop_duplicates().sort_values(by='store_id').copy()

# Menambahkan kolom tambahan ke DataFrame untuk tabel Stores
stores_df['Order Processing'] = df.groupby('store_id')['Order Processing'].max().values
stores_df['Positive Feedback'] = df.groupby('store_id')['Positive Feedback'].max().values
stores_df['Total Feedback'] = df.groupby('store_id')['Total Feedback'].max().values

# Membuat DataFrame untuk tabel Products
products_df = df.drop(columns=['store_id', 'Store Name', 'Store City', 'Order Processing', 'Positive Feedback', 'Total Feedback'])

# Menyimpan DataFrame ke file CSV
stores_df.to_csv('stores.csv', index=False)
products_df.to_csv('products.csv', index=False)

**Saving Cleaned Data:** Finally, we save the cleaned and prepared data into new CSV files for further analysis and usage.

In [39]:
df.to_csv('mechanical-keyboard-full.csv', index = False)